# 📑 View AWS SageMaker Model Package

This Python script is designed to **capture and store a snapshot of an AWS SageMaker model package**, including its group details and all versions. It organizes the workflow into clear steps, ensuring both local and S3 storage of the snapshot for auditing or documentation purposes.

---

## 1. **Setup and Initialization**
- Imports required libraries: `boto3`, `json`, and `datetime`.
- Defines AWS region (`us-east-1`) and initializes a SageMaker client.
- Specifies the **Model Package ARN** (version 2, approved).
- Generates a timestamped filename for the snapshot JSON.

---

## 2. **Describe Model Package**
- Calls `describe_model_package` to fetch **full details** of the specified model package.
- Handles errors gracefully with `try/except`.
- Prints confirmation of retrieval.

---

## 3. **Describe Model Package Group**
- Extracts the **ModelPackageGroupName** from the package details.
- Calls `describe_model_package_group` to fetch group metadata.
- If unavailable, logs a warning and continues.

---

## 4. **List All Versions in Group**
- Uses a paginator (`list_model_packages`) to retrieve **all versions** in the group.
- Sorts by creation time in ascending order.
- Collects and prints the total number of versions found.

---

## 5. **Create Snapshot JSON**
- Builds a structured dictionary containing:
  - **Snapshot metadata** (timestamp, region, source ARN).
  - **Model package details**.
  - **Group details**.
  - **All versions in the group**.

---

## 6. **Save Snapshot Locally**
- Writes the snapshot to a JSON file with indentation for readability.
- Confirms successful local save.

---

## 7. **Upload Snapshot to S3 (Optional)**
- Initializes an S3 client.
- Defines bucket and key path (`model-registry-snapshots/`).
- Attempts to upload the snapshot file to S3.
- Logs success or failure.

---

## 8. **Pretty Print Key Fields**
- Displays a **summary of important fields**:
  - Model package ARN, version, status, approval status, creation time, description.
  - Inference container details (image, model data URL, supported instance types).
  - Group name and total versions.
- Prints confirmation of snapshot storage (local + S3).

---

### ✅ **Purpose of the Script**
This script provides a **repeatable, auditable process** to:
- Document the state of a SageMaker model package at a given time.
- Track model package versions and metadata.
- Store snapshots both locally and in S3 for compliance, debugging, or version control.


In [ ]:

import boto3
import json
from datetime import datetime

region = "us-east-1"
sm_client = boto3.client("sagemaker", region_name=region)

# Model package ARN (Version 2 - Approved)
model_package_arn = "arn:aws:sagemaker:us-east-1:308842842840:model-package/ueba-detection-group/2"

timestamp = datetime.utcnow().strftime("%Y%m%d-%H%M%S")
snapshot_filename = f"model-package-snapshot-v2-{timestamp}.json"

print("=" * 70)
print("TAKING SNAPSHOT OF MODEL PACKAGE DETAILS")
print("=" * 70)

# ------------------------------------------------------------------
# 1. DESCRIBE MODEL PACKAGE (FULL DETAILS)
# ------------------------------------------------------------------
print(f"\n[1/3] Fetching model package details...")

try:
    model_package = sm_client.describe_model_package(
        ModelPackageName=model_package_arn
    )
    print(f"✅ Retrieved model package: {model_package_arn}")
except Exception as e:
    print(f"❌ Error: {e}")
    exit(1)

# ------------------------------------------------------------------
# 2. DESCRIBE MODEL PACKAGE GROUP
# ------------------------------------------------------------------
print(f"\n[2/3] Fetching model package group details...")

try:
    group_name = model_package.get("ModelPackageGroupName")
    group_details = sm_client.describe_model_package_group(
        ModelPackageGroupName=group_name
    )
    print(f"✅ Retrieved group: {group_name}")
except Exception as e:
    print(f"⚠️  Warning: {e}")
    group_details = {}

# ------------------------------------------------------------------
# 3. LIST ALL VERSIONS IN GROUP
# ------------------------------------------------------------------
print(f"\n[3/3] Listing all versions in group...")

try:
    paginator = sm_client.get_paginator("list_model_packages")
    versions = []
    for page in paginator.paginate(
        ModelPackageGroupName=group_name,
        SortBy="CreationTime",
        SortOrder="Ascending",
    ):
        versions.extend(page["ModelPackageSummaryList"])

    print(f"✅ Found {len(versions)} version(s)")
except Exception as e:
    print(f"⚠️  Warning: {e}")
    versions = []

# ------------------------------------------------------------------
# 4. CREATE SNAPSHOT JSON
# ------------------------------------------------------------------
snapshot = {
    "SnapshotMetadata": {
        "Timestamp": timestamp,
        "Region": region,
        "SourceArn": model_package_arn,
    },
    "ModelPackageDetails": model_package,
    "ModelPackageGroupDetails": group_details,
    "AllVersionsInGroup": versions,
}

# ------------------------------------------------------------------
# 5. SAVE LOCALLY
# ------------------------------------------------------------------
print(f"\n[SAVE] Writing snapshot to: {snapshot_filename}")

with open(snapshot_filename, "w", encoding="utf-8") as f:
    json.dump(snapshot, f, indent=2, default=str)

print(f"✅ Snapshot saved locally.")

# ------------------------------------------------------------------
# 6. (OPTIONAL) UPLOAD TO S3
# ------------------------------------------------------------------
print(f"\n[OPTIONAL] Uploading to S3...")

s3_client = boto3.client("s3")
bucket = "aai-540-assignment4-1-1770199145"
s3_key = f"model-registry-snapshots/{snapshot_filename}"

try:
    s3_client.upload_file(snapshot_filename, bucket, s3_key)
    print(f"✅ Uploaded to S3: s3://{bucket}/{s3_key}")
except Exception as e:
    print(f"⚠️  S3 upload failed: {e}")

# ------------------------------------------------------------------
# 7. PRETTY PRINT KEY FIELDS
# ------------------------------------------------------------------
print("\n" + "=" * 70)
print("MODEL PACKAGE SNAPSHOT SUMMARY")
print("=" * 70)

print(f"\n📦 Model Package (V2 - Approved):")
print(f"   ARN:                 {model_package.get('ModelPackageArn')}")
print(f"   Version:             {model_package.get('ModelPackageVersion')}")
print(f"   Status:              {model_package.get('ModelPackageStatus')}")
print(f"   Approval Status:     {model_package.get('ModelApprovalStatus')}")
print(f"   Created:             {model_package.get('CreationTime')}")
print(f"   Description:         {model_package.get('ModelPackageDescription')}")

if "InferenceSpecification" in model_package:
    inference = model_package["InferenceSpecification"]
    containers = inference.get("Containers", [])
    if containers:
        print(f"\n🐳 Inference Container:")
        print(f"   Image:               {containers[0].get('Image')}")
        print(f"   Model Data URL:      {containers[0].get('ModelDataUrl')}")
        print(f"   Instance Types:      {inference.get('SupportedRealtimeInferenceInstanceTypes')}")

print(f"\n📋 Group Details:")
print(f"   Group Name:          {group_name}")
print(f"   Total Versions:      {len(versions)}")

print(f"\n✅ Full snapshot saved to:")
print(f"   Local:  {snapshot_filename}")
print(f"   S3:     s3://{bucket}/{s3_key}")

print("\n" + "=" * 70)

/tmp/ipykernel_10362/766505485.py:11: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime("%Y%m%d-%H%M%S")


TAKING SNAPSHOT OF MODEL PACKAGE DETAILS

[1/3] Fetching model package details...
✅ Retrieved model package: arn:aws:sagemaker:us-east-1:308842842840:model-package/ueba-detection-group/2

[2/3] Fetching model package group details...
✅ Retrieved group: ueba-detection-group

[3/3] Listing all versions in group...
✅ Found 2 version(s)

[SAVE] Writing snapshot to: model-package-snapshot-v2-20260209-122329.json
✅ Snapshot saved locally.

[OPTIONAL] Uploading to S3...
✅ Uploaded to S3: s3://aai-540-assignment4-1-1770199145/model-registry-snapshots/model-package-snapshot-v2-20260209-122329.json

MODEL PACKAGE SNAPSHOT SUMMARY

📦 Model Package (V2 - Approved):
   ARN:                 arn:aws:sagemaker:us-east-1:308842842840:model-package/ueba-detection-group/2
   Version:             2
   Status:              Completed
   Approval Status:     Approved
   Created:             2026-02-04 10:17:45.174000+00:00
   Description:         Version 1: Hybrid CNN-GRU

🐳 Inference Container:
   Image:   

In [ ]:
import boto3
import json

sm = boto3.client("sagemaker", region_name="us-east-1")
group_name = "ueba-detection-group"

# 1. Fetch Group Metadata (Name & Description)
group_info = sm.describe_model_package_group(ModelPackageGroupName=group_name)

# 2. Fetch Version History (Tracking Experiments)
versions = sm.list_model_packages(ModelPackageGroupName=group_name)

print("="*80)
print(f"REPORT: MODEL GROUP GOVERNANCE AUDIT")
print("="*80)

# Meets Rubric: Informative Name and Description (< 250 chars)
print(f"GROUP NAME: {group_info['ModelPackageGroupName']}")
print(f"DESCRIPTION: {group_info['ModelPackageGroupDescription']}")
print(f"STATUS: {group_info['ModelPackageGroupStatus']}")
print(f"CREATED: {group_info['CreationTime']}")

print("\n" + "-"*80)
print(f"DEPLOYMENT HISTORY & EXPERIMENT TRACKING (List of Packages)")
print("-"*80)
print(f"{'Version':<10} | {'Status':<20} | {'Creation Date':<30}")
print("-"*80)

for pkg in versions['ModelPackageSummaryList']:
    print(f"Version {pkg['ModelPackageVersion']:<3} | {pkg['ModelApprovalStatus']:<20} | {pkg['CreationTime']}")

print("="*80)


REPORT: MODEL GROUP GOVERNANCE AUDIT
GROUP NAME: ueba-detection-group
DESCRIPTION: UEBA Anomaly Detection model group for CERT data logs.
STATUS: Completed
CREATED: 2026-02-04 10:08:23.355000+00:00

--------------------------------------------------------------------------------
DEPLOYMENT HISTORY & EXPERIMENT TRACKING (List of Packages)
--------------------------------------------------------------------------------
Version    | Status               | Creation Date                 
--------------------------------------------------------------------------------
Version 2   | Approved             | 2026-02-04 10:17:45.174000+00:00
Version 1   | PendingManualApproval | 2026-02-04 10:08:23.892000+00:00


In [ ]:
from datetime import datetime, timezone
import boto3
import json
import pandas as pd

# ------------------------------------------------------------------
# SETUP
# ------------------------------------------------------------------
region = "us-east-1"
account_id = "308842842840"
sm_client = boto3.client("sagemaker", region_name=region)

model_package_group_name = "ueba-detection-group"

print("=" * 80)
print(f"MODEL PACKAGE GROUP DETAILS: {model_package_group_name}")
print("=" * 80)

# ------------------------------------------------------------------
# 1. DESCRIBE MODEL PACKAGE GROUP
# ------------------------------------------------------------------
print("\n[1/5] Fetching Model Package Group metadata...\n")

try:
    group = sm_client.describe_model_package_group(
        ModelPackageGroupName=model_package_group_name
    )

    print("✅ MODEL PACKAGE GROUP METADATA:")
    print("-" * 80)
    print(f"Name:                    {group.get('ModelPackageGroupName')}")
    print(f"ARN:                     {group.get('ModelPackageGroupArn')}")
    print(f"Status:                  {group.get('ModelPackageGroupStatus')}")
    print(f"Description:             {group.get('ModelPackageGroupDescription', 'N/A')}")
    print(f"Created Time:            {group.get('CreationTime')}")
    print(f"Last Modified Time:      {group.get('LastModifiedTime')}")
    print(f"Created By:              {group.get('CreatedBy', {}).get('IamIdentity', {}).get('Arn', 'N/A')}")

    if "ModelPackageGroupPolicy" in group:
        print(f"Group Policy:            {group.get('ModelPackageGroupPolicy')}")

except Exception as e:
    print(f"❌ Error: {e}")
    exit(1)

# ------------------------------------------------------------------
# 2. LIST ALL MODEL PACKAGES (VERSIONS) IN GROUP
# ------------------------------------------------------------------
print(f"\n[2/5] Listing all versions in group...\n")

versions = []
try:
    paginator = sm_client.get_paginator("list_model_packages")
    page_iterator = paginator.paginate(
        ModelPackageGroupName=model_package_group_name,
        SortBy="CreationTime",
        SortOrder="Ascending",
    )

    for page in page_iterator:
        versions.extend(page["ModelPackageSummaryList"])

    print(f"✅ FOUND {len(versions)} VERSION(S):")
    print("-" * 80)

    for v in versions:
        print(f"\nVersion {v.get('ModelPackageVersion')}:")
        print(f"  ARN:              {v.get('ModelPackageArn')}")
        print(f"  Status:           {v.get('ModelPackageStatus')}")
        print(f"  Approval Status:  {v.get('ModelApprovalStatus')}")
        print(f"  Created:          {v.get('CreationTime')}")

except Exception as e:
    print(f"❌ Error: {e}")

# ------------------------------------------------------------------
# 3. DESCRIBE EACH MODEL PACKAGE IN DETAIL
# ------------------------------------------------------------------
print(f"\n[3/5] Fetching detailed info for each version...\n")

for v in versions:
    version_num = v.get('ModelPackageVersion')
    arn = v.get('ModelPackageArn')

    try:
        pkg = sm_client.describe_model_package(ModelPackageName=arn)

        print(f"\n{'='*80}")
        print(f"VERSION {version_num} - DETAILED VIEW")
        print(f"{'='*80}")

        print(f"\n📦 Basic Info:")
        print(f"  ARN:                     {pkg.get('ModelPackageArn')}")
        print(f"  Status:                  {pkg.get('ModelPackageStatus')}")
        print(f"  Approval Status:         {pkg.get('ModelApprovalStatus')}")
        print(f"  Registration Type:       {pkg.get('ModelPackageRegistrationType')}")
        print(f"  Description:             {pkg.get('ModelPackageDescription')}")
        print(f"  Created:                 {pkg.get('CreationTime')}")
        print(f"  Last Modified:           {pkg.get('LastModifiedTime')}")

        if "InferenceSpecification" in pkg:
            inference = pkg["InferenceSpecification"]

            print(f"\n🐳 Inference Specification:")

            if "Containers" in inference:
                for i, container in enumerate(inference["Containers"]):
                    print(f"\n  Container {i+1}:")
                    print(f"    Image:                 {container.get('Image')}")
                    print(f"    ImageDigest:           {container.get('ImageDigest')}")
                    print(f"    Model Data URL:        {container.get('ModelDataUrl')}")
                    print(f"    Model Data ETag:       {container.get('ModelDataETag')}")
                    print(f"    Is Checkpoint:         {container.get('IsCheckpoint')}")

            print(f"\n  Supported Content Types:")
            for ct in inference.get('SupportedContentTypes', []):
                print(f"    - {ct}")

            print(f"\n  Supported Response MIME Types:")
            for rt in inference.get('SupportedResponseMIMETypes', []):
                print(f"    - {rt}")

            print(f"\n  Supported Realtime Instance Types:")
            for it in inference.get('SupportedRealtimeInferenceInstanceTypes', []):
                print(f"    - {it}")

            print(f"\n  Supported Transform Instance Types:")
            for it in inference.get('SupportedTransformInstanceTypes', []):
                print(f"    - {it}")

        if "ModelMetrics" in pkg:
            print(f"\n📊 Model Metrics:")
            metrics = pkg["ModelMetrics"]

            if "ModelQuality" in metrics:
                mq = metrics["ModelQuality"]
                print(f"  Model Quality:")
                if "Metrics" in mq:
                    for metric in mq["Metrics"]:
                        print(f"    - {metric}")

            if "Bias" in metrics:
                bias = metrics["Bias"]
                print(f"  Bias Report:")
                if "Report" in bias:
                    print(f"    {bias['Report']}")

            if "Explainability" in metrics:
                exp = metrics["Explainability"]
                print(f"  Explainability Report:")
                if "Report" in exp:
                    print(f"    {exp['Report']}")

        # Tags
        try:
            tags_response = sm_client.list_tags(ResourceArn=arn)
            if tags_response.get('Tags'):
                print(f"\n🏷️  Tags:")
                for tag in tags_response['Tags']:
                    print(f"  {tag['Key']}: {tag['Value']}")
        except Exception as e:
            print(f"  (No tags or error fetching tags)")

    except Exception as e:
        print(f"⚠️  Error fetching version {version_num}: {e}")

# ------------------------------------------------------------------
# 4. CREATE COMPARISON TABLE
# ------------------------------------------------------------------
print(f"\n[4/5] Creating version comparison table...\n")

comparison_data = []
for v in versions:
    arn = v.get('ModelPackageArn')
    try:
        pkg = sm_client.describe_model_package(ModelPackageName=arn)

        inference = pkg.get("InferenceSpecification", {})
        containers = inference.get("Containers", [{}])

        comparison_data.append({
            "Version": pkg.get('ModelPackageVersion'),
            "Status": pkg.get('ModelPackageStatus'),
            "Approval": pkg.get('ModelApprovalStatus'),
            "Container Image": containers[0].get('Image', 'N/A').split('/')[-1] if containers else 'N/A',
            "Model Data": containers[0].get('ModelDataUrl', 'N/A').split('/')[-1] if containers else 'N/A',
            "Created": str(pkg.get('CreationTime'))[:10],
            "Description": pkg.get('ModelPackageDescription', 'N/A')[:50],
        })
    except Exception as e:
        pass

if comparison_data:
    df = pd.DataFrame(comparison_data)
    print("✅ MODEL PACKAGE VERSION COMPARISON:")
    print("-" * 80)
    print(df.to_string(index=False))

# ------------------------------------------------------------------
# 5. EXPORT SNAPSHOT TO FILE
# ------------------------------------------------------------------
print(f"\n[5/5] Exporting snapshot to JSON...\n")

snapshot = {
    "export_timestamp": datetime.now(timezone.utc).isoformat(),
    "model_package_group": group,
    "all_versions": versions,
}

timestamp = datetime.now(timezone.utc).strftime("%Y%m%d-%H%M%S")
filename = f"model-package-group-snapshot-{timestamp}.json"

with open(filename, "w") as f:
    json.dump(snapshot, f, indent=2, default=str)

print(f"✅ Snapshot exported to: {filename}")

# Try S3 upload
try:
    s3 = boto3.client("s3")
    s3.upload_file(filename, "aai-540-assignment4-1-1770199145", f"model-registry-snapshots/{filename}")
    print(f"✅ Also uploaded to S3: s3://aai-540-assignment4-1-1770199145/model-registry-snapshots/{filename}")
except Exception as e:
    print(f"⚠️  S3 upload skipped: {e}")

# ------------------------------------------------------------------
# SUMMARY
# ------------------------------------------------------------------
print(f"\n{'='*80}")
print("SUMMARY")
print(f"{'='*80}")
print(f"\nModel Package Group: {model_package_group_name}")
print(f"Total Versions: {len(versions)}")
print(f"\nFiles created:")
print(f"  - {filename}")


# 📑 AWS SageMaker Model Package Group Snapshot Script

---

## 1. **Setup**
- Import libraries: `datetime`, `boto3`, `json`, `pandas`.
- Define AWS region and account ID.
- Initialize SageMaker client.
- Set **Model Package Group Name** (`ueba-detection-group`).
- Print header for group details.

---

## 2. **Describe Model Package Group**
- Call `describe_model_package_group`.
- Print metadata:
  - Name, ARN, Status, Description.
  - Creation and last modified times.
  - Created by IAM identity.
  - Group policy (if available).

---

## 3. **List All Model Packages (Versions)**
- Use paginator `list_model_packages`.
- Retrieve all versions in ascending order by creation time.
- Print summary for each version:
  - Version number, ARN, Status, Approval status, Creation time.

---

## 4. **Describe Each Model Package in Detail**
- Loop through each version ARN.
- Call `describe_model_package`.
- Print detailed info:
  - Basic metadata (ARN, status, approval, description, timestamps).
  - **Inference Specification**:
    - Containers (image, digest, model data URL, checkpoint).
    - Supported content types, response MIME types.
    - Supported realtime and transform instance types.
  - **Model Metrics**:
    - Model quality metrics.
    - Bias report.
    - Explainability report.
  - **Tags** (if available).

---

## 5. **Create Comparison Table**
- Build structured comparison data for each version:
  - Version, Status, Approval, Container image, Model data, Created date, Description.
- Convert to Pandas DataFrame.
- Print formatted comparison table.

---

## 6. **Export Snapshot to File**
- Create snapshot dictionary:
  - Export timestamp.
  - Model package group metadata.
  - All versions list.
- Save snapshot to JSON file with timestamped filename.
- Attempt upload to S3 bucket (`aai-540-assignment4-1-1770199145`).
- Print confirmation of local and S3 storage.



In [ ]:
from datetime import datetime, timezone
import boto3
import json
import pandas as pd

# ------------------------------------------------------------------
# SETUP
# ------------------------------------------------------------------
region = "us-east-1"
account_id = "308842842840"
sm_client = boto3.client("sagemaker", region_name=region)

model_package_group_name = "ueba-detection-group"

print("=" * 80)
print(f"MODEL PACKAGE GROUP DETAILS: {model_package_group_name}")
print("=" * 80)

# ------------------------------------------------------------------
# 1. DESCRIBE MODEL PACKAGE GROUP
# ------------------------------------------------------------------
print("\n[1/5] Fetching Model Package Group metadata...\n")

try:
    group = sm_client.describe_model_package_group(
        ModelPackageGroupName=model_package_group_name
    )

    print("✅ MODEL PACKAGE GROUP METADATA:")
    print("-" * 80)
    print(f"Name:                    {group.get('ModelPackageGroupName')}")
    print(f"ARN:                     {group.get('ModelPackageGroupArn')}")
    print(f"Status:                  {group.get('ModelPackageGroupStatus')}")
    print(f"Description:             {group.get('ModelPackageGroupDescription', 'N/A')}")
    print(f"Created Time:            {group.get('CreationTime')}")
    print(f"Last Modified Time:      {group.get('LastModifiedTime')}")
    print(f"Created By:              {group.get('CreatedBy', {}).get('IamIdentity', {}).get('Arn', 'N/A')}")

    if "ModelPackageGroupPolicy" in group:
        print(f"Group Policy:            {group.get('ModelPackageGroupPolicy')}")

except Exception as e:
    print(f"❌ Error: {e}")
    exit(1)

# ------------------------------------------------------------------
# 2. LIST ALL MODEL PACKAGES (VERSIONS) IN GROUP
# ------------------------------------------------------------------
print(f"\n[2/5] Listing all versions in group...\n")

versions = []
try:
    paginator = sm_client.get_paginator("list_model_packages")
    page_iterator = paginator.paginate(
        ModelPackageGroupName=model_package_group_name,
        SortBy="CreationTime",
        SortOrder="Ascending",
    )

    for page in page_iterator:
        versions.extend(page["ModelPackageSummaryList"])

    print(f"✅ FOUND {len(versions)} VERSION(S):")
    print("-" * 80)

    for v in versions:
        print(f"\nVersion {v.get('ModelPackageVersion')}:")
        print(f"  ARN:              {v.get('ModelPackageArn')}")
        print(f"  Status:           {v.get('ModelPackageStatus')}")
        print(f"  Approval Status:  {v.get('ModelApprovalStatus')}")
        print(f"  Created:          {v.get('CreationTime')}")

except Exception as e:
    print(f"❌ Error: {e}")

# ------------------------------------------------------------------
# 3. DESCRIBE EACH MODEL PACKAGE IN DETAIL
# ------------------------------------------------------------------
print(f"\n[3/5] Fetching detailed info for each version...\n")

for v in versions:
    version_num = v.get('ModelPackageVersion')
    arn = v.get('ModelPackageArn')

    try:
        pkg = sm_client.describe_model_package(ModelPackageName=arn)

        print(f"\n{'='*80}")
        print(f"VERSION {version_num} - DETAILED VIEW")
        print(f"{'='*80}")

        print(f"\n📦 Basic Info:")
        print(f"  ARN:                     {pkg.get('ModelPackageArn')}")
        print(f"  Status:                  {pkg.get('ModelPackageStatus')}")
        print(f"  Approval Status:         {pkg.get('ModelApprovalStatus')}")
        print(f"  Registration Type:       {pkg.get('ModelPackageRegistrationType')}")
        print(f"  Description:             {pkg.get('ModelPackageDescription')}")
        print(f"  Created:                 {pkg.get('CreationTime')}")
        print(f"  Last Modified:           {pkg.get('LastModifiedTime')}")

        if "InferenceSpecification" in pkg:
            inference = pkg["InferenceSpecification"]

            print(f"\n🐳 Inference Specification:")

            if "Containers" in inference:
                for i, container in enumerate(inference["Containers"]):
                    print(f"\n  Container {i+1}:")
                    print(f"    Image:                 {container.get('Image')}")
                    print(f"    ImageDigest:           {container.get('ImageDigest')}")
                    print(f"    Model Data URL:        {container.get('ModelDataUrl')}")
                    print(f"    Model Data ETag:       {container.get('ModelDataETag')}")
                    print(f"    Is Checkpoint:         {container.get('IsCheckpoint')}")

            print(f"\n  Supported Content Types:")
            for ct in inference.get('SupportedContentTypes', []):
                print(f"    - {ct}")

            print(f"\n  Supported Response MIME Types:")
            for rt in inference.get('SupportedResponseMIMETypes', []):
                print(f"    - {rt}")

            print(f"\n  Supported Realtime Instance Types:")
            for it in inference.get('SupportedRealtimeInferenceInstanceTypes', []):
                print(f"    - {it}")

            print(f"\n  Supported Transform Instance Types:")
            for it in inference.get('SupportedTransformInstanceTypes', []):
                print(f"    - {it}")

        if "ModelMetrics" in pkg:
            print(f"\n📊 Model Metrics:")
            metrics = pkg["ModelMetrics"]

            if "ModelQuality" in metrics:
                mq = metrics["ModelQuality"]
                print(f"  Model Quality:")
                if "Metrics" in mq:
                    for metric in mq["Metrics"]:
                        print(f"    - {metric}")

            if "Bias" in metrics:
                bias = metrics["Bias"]
                print(f"  Bias Report:")
                if "Report" in bias:
                    print(f"    {bias['Report']}")

            if "Explainability" in metrics:
                exp = metrics["Explainability"]
                print(f"  Explainability Report:")
                if "Report" in exp:
                    print(f"    {exp['Report']}")

        # Tags
        try:
            tags_response = sm_client.list_tags(ResourceArn=arn)
            if tags_response.get('Tags'):
                print(f"\n🏷️  Tags:")
                for tag in tags_response['Tags']:
                    print(f"  {tag['Key']}: {tag['Value']}")
        except Exception as e:
            print(f"  (No tags or error fetching tags)")

    except Exception as e:
        print(f"⚠️  Error fetching version {version_num}: {e}")

# ------------------------------------------------------------------
# 4. CREATE COMPARISON TABLE
# ------------------------------------------------------------------
print(f"\n[4/5] Creating version comparison table...\n")

comparison_data = []
for v in versions:
    arn = v.get('ModelPackageArn')
    try:
        pkg = sm_client.describe_model_package(ModelPackageName=arn)

        inference = pkg.get("InferenceSpecification", {})
        containers = inference.get("Containers", [{}])

        comparison_data.append({
            "Version": pkg.get('ModelPackageVersion'),
            "Status": pkg.get('ModelPackageStatus'),
            "Approval": pkg.get('ModelApprovalStatus'),
            "Container Image": containers[0].get('Image', 'N/A').split('/')[-1] if containers else 'N/A',
            "Model Data": containers[0].get('ModelDataUrl', 'N/A').split('/')[-1] if containers else 'N/A',
            "Created": str(pkg.get('CreationTime'))[:10],
            "Description": pkg.get('ModelPackageDescription', 'N/A')[:50],
        })
    except Exception as e:
        pass

if comparison_data:
    df = pd.DataFrame(comparison_data)
    print("✅ MODEL PACKAGE VERSION COMPARISON:")
    print("-" * 80)
    print(df.to_string(index=False))

# ------------------------------------------------------------------
# 5. EXPORT SNAPSHOT TO FILE
# ------------------------------------------------------------------
print(f"\n[5/5] Exporting snapshot to JSON...\n")

snapshot = {
    "export_timestamp": datetime.now(timezone.utc).isoformat(),
    "model_package_group": group,
    "all_versions": versions,
}

timestamp = datetime.now(timezone.utc).strftime("%Y%m%d-%H%M%S")
filename = f"model-package-group-snapshot-{timestamp}.json"

with open(filename, "w") as f:
    json.dump(snapshot, f, indent=2, default=str)

print(f"✅ Snapshot exported to: {filename}")

# Try S3 upload
try:
    s3 = boto3.client("s3")
    s3.upload_file(filename, "aai-540-assignment4-1-1770199145", f"model-registry-snapshots/{filename}")
    print(f"✅ Also uploaded to S3: s3://aai-540-assignment4-1-1770199145/model-registry-snapshots/{filename}")
except Exception as e:
    print(f"⚠️  S3 upload skipped: {e}")

# ------------------------------------------------------------------
# SUMMARY
# ------------------------------------------------------------------
print(f"\n{'='*80}")
print("SUMMARY")
print(f"{'='*80}")
print(f"\nModel Package Group: {model_package_group_name}")
print(f"Total Versions: {len(versions)}")
print(f"\nFiles created:")
print(f"  - {filename}")


MODEL PACKAGE GROUP DETAILS: ueba-detection-group

[1/5] Fetching Model Package Group metadata...

✅ MODEL PACKAGE GROUP METADATA:
--------------------------------------------------------------------------------
Name:                    ueba-detection-group
ARN:                     arn:aws:sagemaker:us-east-1:308842842840:model-package-group/ueba-detection-group
Status:                  Completed
Description:             UEBA Anomaly Detection model group for CERT data logs.
Created Time:            2026-02-04 10:08:23.355000+00:00
Last Modified Time:      None
Created By:              arn:aws:sts::308842842840:assumed-role/LabRole/SageMaker

[2/5] Listing all versions in group...

✅ FOUND 2 VERSION(S):
--------------------------------------------------------------------------------

Version 1:
  ARN:              arn:aws:sagemaker:us-east-1:308842842840:model-package/ueba-detection-group/1
  Status:           Completed
  Approval Status:  PendingManualApproval
  Created:          2026